In [18]:
# -*- coding: utf-8 -*-

import urllib
import urllib2
import re
from bs4 import BeautifulSoup
import sys
import os
import errno
import random
import string
from argparse import ArgumentParser
import time
from six.moves.urllib.parse import urlencode
import json 

def entryurl_googleimage(url, offset):
    splitkey1 = ''
    splitkey2 = ''
    if 'ijn=1'in url and 'start=100' in url:
        splitkey1 = 'ijn=1'
        splitkey2 = 'start=100'
    elif 'ijn=2' in url and 'start=200' in url:
        splitkey1 = 'ijn=2'
        splitkey2 = 'start=200'
    elif 'ijn=3' in url and 'start=300' in url:
        splitkey1 = 'ijn=3'
        splitkey2 = 'start=300'
    else:
        print 'ijn error'

    url = url.split(splitkey1)
    url = url = '{}ijn={}{}'.format(url[0],offset//100,url[1])
    
    url = url.split(splitkey2)
    url = url = '{}start={}{}'.format(url[0],offset,url[1])
    
    return url
    
def response_contents(url):
    headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36'}
    request = urllib2.Request(url, headers = headers)
    try:
        response = urllib2.urlopen(request)
        content = response.read()
    except urllib2.HTTPError as err:
        content = ''
        if err.code == 404:
            print 'HTTP Error 404: Not Found'
        else:
            raise

    return content

def extract_infos_googleimage(content):
    img_id_url_dict = dict()
    
    image_divs = BeautifulSoup(content, 'lxml').find_all('div', class_='rg_meta')
    soup = BeautifulSoup(content, 'lxml').select('.rg_di.rg_bx.rg_el.ivg-i .rg_meta')
    for div in image_divs:
        try:
            meta = json.loads(div.text)
        except ValueError:
            print("Oops!  That was No JSON object could be decoded.")
            continue
        if 'ou' in meta and 'id' in meta:
            if meta['id'] not in img_id_url_dict:
                img_id_url_dict[meta['id']] = meta['ou']
            
    return img_id_url_dict

# def extract_infos_googleimage(content):
#     img_id_url_dict = dict()
#     soup = BeautifulSoup(content, 'lxml')
#     meta = str(json.loads(soup.text))
#     soup = BeautifulSoup(meta, 'lxml').select('.rg_meta')
    
#     ou_pattern = re.compile(r'"ou":"(.*?)","ow"')
#     id_pattern = re.compile(r'"id":"(.*?):","isu"')
#     ity_pattern = re.compile(r'"ity":"(.*?)","oh"')
#     url_pattern = re.compile(r'(.*?).jpg')
    
#     for meta in soup:
#         meta = str(meta)
#         ou_find = re.findall(ou_pattern, meta)
#         id_find = re.findall(id_pattern, meta)
#         if len(ou_find)==0 or len(id_find)==0:
#             continue
#         ou_find = ou_find[0]
#         imgid = id_find[0]
#         ity_find = re.findall(ity_pattern, meta)
#         if len(ity_find):
#             url_find = re.findall(url_pattern, ou_find)
#             if len(url_find)==0:
#                 continue
#             imgurl = url_find[0] + '.jpg'
#         if imgid not in img_id_url_dict:
#             img_id_url_dict[imgid] = imgurl
            
#     return img_id_url_dict

def save_infos(img_id_url_dict, idprefix, fname):
    fid = open(fname, 'w')

    for imgid in img_id_url_dict:
        imgurl = img_id_url_dict[imgid]
        try:
            fid.write(idprefix+ '_' + str(imgid) + '   ' + imgurl + '\n')
        except Exception,e:
            print Exception,":",e

    fid.close()

def _mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5 (except OSError, exc: for Python <2.5)
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
def main(keyword, initurl):
    offset = 0
    max_num = 1000
    idprefix='googleimage'
    extension = 'txt'
    outdir = '/home/lpzhang/Desktop/crawler/GoogleImages'
    
    img_dir = os.path.join(outdir, '_'.join(keyword.split()))
    _mkdir_p(img_dir)
    assert os.path.exists(img_dir), 'img_dir not exists'

    outfile= os.path.join(img_dir, '{}_{}_{}.{}'.format(idprefix, '_'.join(keyword.split()), len(os.listdir(img_dir)), extension))
    print outfile

    image_id_url_dict = dict()
    for i in range(offset, offset + max_num, 100):
        print 'start from', i
        url = entryurl_googleimage(initurl, i)
        pages =  response_contents(url)
        img_id_url_dict = extract_infos_googleimage(pages)

        image_id_url_dict.update(img_id_url_dict)

    print 'save image_id_url_dict'
    save_infos(image_id_url_dict, idprefix, outfile)

###############
keyword = ''
urllist = list()

# urllist.append('')
# urllist.append('')

# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')


for initurl in urllist:
    main(keyword, initurl)
    
print '########### done ##############'



/home/lpzhang/Desktop/crawler/GoogleImages/chime/googleimage_chime_13.txt
start from 0
start from 100
start from 200
start from 300
start from 400
start from 500
start from 600
start from 700
start from 800
start from 900
save image_id_url_dict
/home/lpzhang/Desktop/crawler/GoogleImages/chime/googleimage_chime_14.txt
start from 0
start from 100
start from 200
start from 300
start from 400
start from 500
start from 600
start from 700
start from 800
start from 900
save image_id_url_dict
/home/lpzhang/Desktop/crawler/GoogleImages/chime/googleimage_chime_15.txt
start from 0
start from 100
start from 200
start from 300
start from 400
start from 500
start from 600
start from 700
start from 800
start from 900
save image_id_url_dict
/home/lpzhang/Desktop/crawler/GoogleImages/chime/googleimage_chime_16.txt
start from 0
start from 100
start from 200
start from 300
start from 400
start from 500
start from 600
start from 700
start from 800
start from 900
save image_id_url_dict
/home/lpzhang/Deskto